## Объединение данных

Распутывать все эти таблички начнем с таблицы аэропорта

In [185]:
import re
import json
import csv
import pandas as pd
from pprint import pprint
import numpy as np

In [186]:
class Airport:
    max_id = 0
    def __init__(self, iata, country, city):
        Airport.max_id += 1
        self.id = Airport.max_id
        self.iata = iata
        self.country = country
        self.city = city
    
    def __eq__(self, other):
        return self.iata == other.iata
    def __hash__(self):
        return hash(self.iata)
    def __str__(self):
        return '{id: %s, iata: %s, country: %s, city: %s}' % (self.id, self.iata, self.country, self.city)

In [187]:
class Flight:
    max_id = 0
    def __init__(self, name, date, airport_from, airport_to):
        Flight.max_id += 1
        self.id = Flight.max_id
        self.persons = set([])
        self.name = name
        self.date = date
        self.airport_from = airport_from
        self.airport_to = airport_to
    
    def __eq__(self, other):
        return (self.name == other.name 
                and self.date == other.date 
                and self.airport_from == other.airport_from 
                and self.airport_to == other.airport_to)
    def __hash__(self):
        return hash(self.name) ^ hash(self.date) ^ hash(self.airport_from) ^ hash(self.airport_to) 
    def __str__(self):
        return '{id: %s, name: %s, date: %s, airport_from: %s, airport_to: %s, persons: %s}' % (
            self.id, 
            self.name, 
            self.date, 
            self.airport_from, 
            self.airport_to, 
            list(self.persons)
        )
    

In [188]:
class Person:
    max_id = 0
    def __init__(self, document, first_name, last_name, uuid=None):
        Person.max_id += 1
        self.id = Person.max_id
        self.flights = set([])
        self.cards = set([])
        self.document = document
        self.first_name = first_name
        self.last_name = last_name
        self.uuid = uuid
        
    def __eq__(self, other):
        if self.document and other.document:
            return self.document == other.document
        else:
            return self.uuid == other.uuid
    def __hash__(self):
        if self.document:
            return hash(self.document)
        else:
            return hash(self.uuid)
    def __str__(self):
        return '{id: %s, document: %s, first_name: %s, last_name: %s, flights: %s}' % (
            self.id, 
            self.document, 
            self.first_name, 
            self.last_name,
            list(self.flights)
        )

In [189]:
class Card:
    max_id = 0
    def __init__(self, status, number):
        Card.max_id += 1
        self.id = Card.max_id
        self.persons = set([])
        self.status = status
        self.number = number
    
    def __eq__(self, other):
        return self.number == other.number
    def __hash__(self):
        return hash(self.number)
    def __str__(self):
        return 'id: %s, status: %s, number: %s, persons: %s' % (
            self.id,
            self.status,
            self.number,
            self.persons
        )

In [190]:
airports = {}
def get_airport_id(iata, country=None, city=None):
    global airports
    
    airport = Airport(iata, country, city)
    if airport in airports:
        Airport.max_id -= 1
        airport = airports[airport]
        return airport.id, airport
    
    airports[airport] = airport
    return Airport.max_id, airport

In [191]:
flights = {}
def get_flight_id(name, date, iata_from, iata_to):
    global flights
    
    airport_from_id, _ = get_airport_id(iata_from)
    airport_to_id, _ = get_airport_id(iata_to)
    
    flight = Flight(name, date, airport_from_id, airport_to_id)
    if flight in flights:
        Flight.max_id -= 1
        flight = flights[flight]
        return flight.id, flight
    
    flights[flight] = flight
    return Flight.max_id, flight

In [192]:
persons = {}
def get_person_id(document, first_name, last_name, uuid=None):
    global persons
    
    person = Person(document, first_name, last_name, uuid)
    if person in persons:
        Person.max_id -= 1
        person = persons[person]
        return person.id, person
    
    persons[person] = person
    return Person.max_id, person

In [193]:
cards = {}
def get_card_id(status, number):
    global cards
    
    card = Card(status, number)
    if card in cards:
        Card.max_id -= 1
        card = cards[card]
        return card.id, card
    
    cards[card] = card
    return card.id, card

In [194]:
Airport.max_id = 0
Flight.max_id = 0
Person.max_id = 0
Card.max_id = 0
airports = {}
flights = {}
persons = {}
cards = {}

map_person = {}
map_airport = {}
# SkyTeamExchange_flight = pd.read_csv('./SkyTeam-Exchange/csv/SkyTeam-Exchange.flight.csv')

FrequentFlyerForumProfiles_place = pd.read_csv('./FrequentFlyerForum-Profiles/csv/FrequentFlyerForum-Profiles.place.csv')
FrequentFlyerForumProfiles_flight = pd.read_csv('./FrequentFlyerForum-Profiles/csv/FrequentFlyerForum-Profiles.flight.csv')
FrequentFlyerForumProfiles_account = pd.read_csv('./FrequentFlyerForum-Profiles/csv/FrequentFlyerForum-Profiles.account.csv')
FrequentFlyerForumProfiles_account.index = FrequentFlyerForumProfiles_account.id
FrequentFlyerForumProfiles_loyality = pd.read_csv('./FrequentFlyerForum-Profiles/csv/FrequentFlyerForum-Profiles.loyality.csv')
acc_len = len(FrequentFlyerForumProfiles_account.id)

for index, place in FrequentFlyerForumProfiles_place.iterrows():
    airport_id, airport_obj = get_airport_id(place.airport, place.country, place.city)
    map_airport[place.id] = place.airport
    
    if index % 100 == 0:
        print('AIRPORT', index, airport_obj)

for index, flight in FrequentFlyerForumProfiles_flight.iterrows():
    flight_id, flight_obj = get_flight_id(
        flight.flight, 
        flight.date, 
        map_airport[flight.departure], 
        map_airport[flight.arrival]
    )
    account_obj = FrequentFlyerForumProfiles_account.loc[flight.account_id]
    person_id, person_obj = get_person_id(
        None, 
        account_obj.first_name, 
        account_obj.last_name, 
        str(account_obj.id)
    )
    
    flight_obj.persons.add(person_id)
    person_obj.flights.add(flight_id)    
    map_person[flight.account_id] = person_id

    if index % 10000 == 0:
        print('FLIGHT', index, str(flight_obj))

for index, loyality in FrequentFlyerForumProfiles_loyality.iterrows():    
    person_id, person_obj = get_person_id(None, None, None, str(map_person[loyality.account_id]))
    card_id, card_obj = get_card_id(loyality.status, loyality.number)
    card_obj.persons.add(person_id)
    person_obj.cards.add(card_id)
    
    if index%1000==0:
        print('CARD', index, card_obj)

AIRPORT 0 {id: 1, iata: MYR, country: nan, city: MYR}
AIRPORT 100 {id: 101, iata: RUN, country: nan, city: Saint Denis De La Reunion}
AIRPORT 200 {id: 201, iata: SWA, country: China, city: Shantou}
AIRPORT 300 {id: 301, iata: MMK, country: Russian Federation, city: Murmansk}
AIRPORT 400 {id: 401, iata: BIO, country: Spain and Canary Islands, city: Bilbao}
AIRPORT 500 {id: 501, iata: RAE, country: Saudi Arabia, city: Arar}
AIRPORT 600 {id: 601, iata: HAH, country: Comoros, city: Moroni}
AIRPORT 700 {id: 701, iata: PSS, country: Argentina, city: Posadas}
AIRPORT 800 {id: 801, iata: BIL, country: United States of, city: Billings MT}
AIRPORT 900 {id: 901, iata: SBN, country: United States of, city: South Bend IN}
AIRPORT 1000 {id: 1001, iata: PBC, country: Mexico, city: Puebla}
AIRPORT 1100 {id: 1101, iata: KEJ, country: Russian Federation, city: Kemerovo}
FLIGHT 0 {id: 1, name: DL3377, date: 2017-03-04, airport_from: 438, airport_to: 668, persons: [1]}
FLIGHT 10000 {id: 9944, name: DL5809

FLIGHT 650000 {id: 572898, name: MU5030, date: 2017-05-31, airport_from: 581, airport_to: 957, persons: [33073]}
FLIGHT 660000 {id: 581036, name: DL3762, date: 2017-07-29, airport_from: 697, airport_to: 230, persons: [33588]}
FLIGHT 670000 {id: 589156, name: DL2845, date: 2017-07-06, airport_from: 538, airport_to: 1125, persons: [34093]}
FLIGHT 680000 {id: 597182, name: MU2048, date: 2017-07-18, airport_from: 1113, airport_to: 182, persons: [34601]}
FLIGHT 690000 {id: 605210, name: DL151, date: 2017-08-25, airport_from: 1125, airport_to: 644, persons: [35113]}
FLIGHT 700000 {id: 613182, name: AM2628, date: 2017-10-29, airport_from: 475, airport_to: 785, persons: [35627]}
FLIGHT 710000 {id: 621171, name: DL5743, date: 2017-12-02, airport_from: 417, airport_to: 557, persons: [36137]}
FLIGHT 720000 {id: 629119, name: SU1436, date: 2017-02-03, airport_from: 794, airport_to: 486, persons: [36640]}
FLIGHT 730000 {id: 637083, name: SU1959, date: 2017-04-18, airport_from: 891, airport_to: 794,

CARD 6000 id: 6001, status: Elite, number: KE 931632626, persons: {4471}
CARD 7000 id: 7001, status: Elite, number: DT 263882403, persons: {5209}
CARD 8000 id: 8001, status: Elite, number: DT 321068189, persons: {5966}
CARD 9000 id: 9001, status: Elite+, number: DT 740730790, persons: {6733}
CARD 10000 id: 10001, status: Elite+, number: FB 479427904, persons: {7490}
CARD 11000 id: 11001, status: Elite+, number: SU 100287260, persons: {8239}
CARD 12000 id: 12001, status: Basic, number: KE 450261892, persons: {8982}
CARD 13000 id: 13001, status: Elite, number: SU 688892383, persons: {9746}
CARD 14000 id: 14001, status: Basic, number: KE 628207968, persons: {10542}
CARD 15000 id: 15001, status: Elite+, number: KE 373106023, persons: {11311}
CARD 16000 id: 16001, status: Basic, number: SU 929269999, persons: {12072}
CARD 17000 id: 17001, status: Basic, number: SU 56306249, persons: {12799}
CARD 18000 id: 18001, status: Basic, number: KE 429493787, persons: {13546}
CARD 19000 id: 19001, sta

In [196]:
SkyTeamExchange_flight = pd.read_csv('./SkyTeam-Exchange/csv/SkyTeam-Exchange.flight.csv')

start = Flight.max_id
for index, flight_csv in SkyTeamExchange_flight.iterrows():
    card_obj = Card(None, flight_csv.card_id)
    Card.max_id -= 1
    
    if card_obj in cards:
        card_obj= cards[card_obj]
        person_id, person_obj = get_person_id(None, None, None, str(list(card_obj.persons)[0]))
        
        flight_id, flight_obj = get_flight_id(
            flight_csv.id, 
            flight_csv.date,
            flight_csv['from'],
            flight_csv['to'])
        flight_obj.persons.add(person_id)
        person_obj.flights.add(flight_id)
        
    if index % 10000 == 0:
        print('SE FLIGHT', index)
print(Flight.max_id - start)

SE FLIGHT 0
SE FLIGHT 10000
SE FLIGHT 20000
SE FLIGHT 30000
SE FLIGHT 40000
SE FLIGHT 50000
SE FLIGHT 60000
SE FLIGHT 70000
SE FLIGHT 80000
SE FLIGHT 90000
SE FLIGHT 100000
SE FLIGHT 110000
SE FLIGHT 120000
SE FLIGHT 130000
SE FLIGHT 140000
SE FLIGHT 150000
SE FLIGHT 160000
SE FLIGHT 170000
SE FLIGHT 180000
SE FLIGHT 190000
SE FLIGHT 200000
SE FLIGHT 210000
SE FLIGHT 220000
SE FLIGHT 230000
SE FLIGHT 240000
SE FLIGHT 250000
SE FLIGHT 260000
SE FLIGHT 270000
SE FLIGHT 280000
SE FLIGHT 290000
SE FLIGHT 300000
SE FLIGHT 310000
SE FLIGHT 320000
SE FLIGHT 330000
SE FLIGHT 340000
SE FLIGHT 350000
SE FLIGHT 360000
SE FLIGHT 370000
SE FLIGHT 380000
SE FLIGHT 390000
SE FLIGHT 400000
SE FLIGHT 410000
SE FLIGHT 420000
SE FLIGHT 430000
SE FLIGHT 440000
SE FLIGHT 450000
SE FLIGHT 460000
SE FLIGHT 470000
SE FLIGHT 480000
SE FLIGHT 490000
SE FLIGHT 500000
SE FLIGHT 510000
SE FLIGHT 520000
SE FLIGHT 530000
SE FLIGHT 540000
SE FLIGHT 550000
SE FLIGHT 560000
SE FLIGHT 570000
SE FLIGHT 580000
SE FLIGHT 5

In [197]:
len(persons.keys())

66592

In [198]:
airports_arr = []
airports_arr.append([
    'id',
    'iata',
    'country',
    'city'
])
for k in airports.keys():
    airports_arr.append([
        k.id, k.iata, k.country, k.city
    ])
airports_df = pd.DataFrame(airports_arr)
airports_df.to_csv('./rel_csv/airports.csv', index=False, header=False)

flights_arr = []
flights_arr.append([
    'id',
    'persons',
    'name',
    'date',
    'airport_from',
    'airport_to'
])
for k in flights.keys():
    flights_arr.append([
        k.id, k.persons, k.name, k.date, k.airport_from, k.airport_to
    ])
flights_df = pd.DataFrame(flights_arr)
flights_df.to_csv('./rel_csv/flights.csv', index=False, header=False)

persons_arr = []
persons_arr.append([
    'id',
    'flights',
    'cards',
    'document',
    'first_name',
    'last_name',
    'uuid'
])
for k in persons:
    persons_arr.append([
        k.id, k.flights, k.cards, k.document, k.first_name, k.last_name, k.uuid
    ])
persons_df = pd.DataFrame(persons_arr)
persons_df.to_csv('./rel_csv/persons.csv', index=False, header=False)

cards_arr = []
cards_arr.append([
    'id',
    'persons',
    'status',
    'number'
])
for k in cards:
    cards_arr.append([
        k.id, k.persons, k.status, k.number
    ])
cards_df = pd.DataFrame(cards_arr)
cards_df.to_csv('./rel_csv/cards.csv', index=False, header=False)

{id: 299, iata: LAX, country: United States of, city: Los Angeles CA}


In [88]:
FrequentFlyerForumProfiles_account.loc[47864]

id               47864
sex             Female
first_name      OLESYA
last_name     GOLOVINA
Name: 47864, dtype: object

In [16]:
FrequentFlyerForumProfiles_flight.head()

,id,account_id,flight,date,codeshare,arrival,departure
0,1,1,DL3377,2017-03-04,False,668,438
1,2,1,DL3630,2017-07-21,False,1006,786
2,3,1,DL5361,2017-03-22,False,697,668
3,4,1,SU3702,2017-01-04,True,440,794
4,5,1,DL3412,2017-08-08,False,644,1105


In [19]:
FrequentFlyerForumProfiles_account.head()

,id,sex,first_name,last_name
id,,,,
1,1,Male,NaN,NaN
2,2,Male,NAZAR,ZYKOV
3,3,NaN,NaN,NaN
4,4,NaN,NaN,NaN
5,5,Male,RUSLAN,D'YAKOV


In [39]:
len(set([1,2]))

2

In [94]:
a, b = get_person_id(None, None, None, 'FrequentFlyer' + str(2))
print(str(b))

{id: 2, document: None, first_name: NAZAR, last_name: ZYKOV, flights: [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 23, 24, 25, 26, 27, 28, 29, 30, 31]}
